# TODO

add condition of translation into indices dataset

check if rpos_vels -> downsample -> rpods_vel -> rpos works

cross: music, ttext, prepend (text)? , ass translation 

In [1]:
# setting device on GPU if available, else CPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

# Additional Info when using cuda
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    print("Memory Usage:")
    print("Allocated:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), "GB")
    print("Cached:   ", round(torch.cuda.memory_reserved(0) / 1024**3, 1), "GB")

Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from functools import partial
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [4]:
def findAllFile(base):
    """
    Recursively find all files in the specified directory.

    Args:
        base (str): The base directory to start the search.

    Returns:
        list: A list of file paths found in the directory and its subdirectories.
    """
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path

In [5]:
from utils.motion_processing.hml_process import recover_from_ric, recover_root_rot_pos
import utils.vis_utils.plot_3d_global as plot_3d
import matplotlib.pyplot as plt

In [55]:
from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser

from configs.config import cfg, get_cfg_defaults
from configs.config_streaming import get_cfg_defaults as strm_get_cfg_defaults


In [56]:
from core.datasets.vq_dataset import VQSMPLXMotionDataset
from core.datasets.vq_dataset import load_dataset as load_dataset_vq
from core.datasets.vq_dataset import simple_collate as simple_collate_vq


In [95]:
dset = VQSMPLXMotionDataset("choreomaster" , "/srv/hays-lab/scratch/sanisetty3/motionx" ,window_size=300, motion_rep = "body", split = "train" , hml_rep = "rv" , fps = 30 )


Total number of motions choreomaster: 34


In [96]:
inp = dset[1]
motion = inp["motion"]

In [97]:
inp["name"]

'choreomaster/1166.npy'

In [101]:

dset.render_hml(copy.deepcopy(motion) , save_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/choreo_ztzor.gif", from_rotation = True, zero_trans = True, zero_orient = True)


rpos tensor([[0.0000, 0.9785, 0.0000],
        [0.0000, 0.9679, 0.0000],
        [0.0000, 0.9591, 0.0000],
        [0.0000, 0.9522, 0.0000],
        [0.0000, 0.9473, 0.0000],
        [0.0000, 0.9442, 0.0000],
        [0.0000, 0.9423, 0.0000],
        [0.0000, 0.9427, 0.0000],
        [0.0000, 0.9457, 0.0000],
        [0.0000, 0.9505, 0.0000],
        [0.0000, 0.9574, 0.0000],
        [0.0000, 0.9659, 0.0000],
        [0.0000, 0.9765, 0.0000],
        [0.0000, 0.9894, 0.0000],
        [0.0000, 1.0055, 0.0000],
        [0.0000, 1.0239, 0.0000],
        [0.0000, 1.0418, 0.0000],
        [0.0000, 1.0592, 0.0000],
        [0.0000, 1.0762, 0.0000],
        [0.0000, 1.0926, 0.0000],
        [0.0000, 1.1070, 0.0000],
        [0.0000, 1.1176, 0.0000],
        [0.0000, 1.1246, 0.0000],
        [0.0000, 1.1288, 0.0000],
        [0.0000, 1.1309, 0.0000],
        [0.0000, 1.1322, 0.0000],
        [0.0000, 1.1336, 0.0000],
        [0.0000, 1.1355, 0.0000],
        [0.0000, 1.1380, 0.0000],
        [

In [47]:
motion.tensor()

In [48]:
motion.root_params[0]

tensor([0.0000, 0.0000, 0.0000, 0.5070])

In [49]:
r_rot_quat, r_pos = dset.mot2pos.recover_root_rot_pos(motion)

In [51]:
motion = inp["motion"]
motion.root_params[: , :3] = 0
pos = dset.mot2pos.recover_from_ric(motion)

rpos tensor([[0.0000, 0.5070, 0.0000],
        [0.0000, 0.5064, 0.0000],
        [0.0000, 0.5052, 0.0000],
        [0.0000, 0.5051, 0.0000],
        [0.0000, 0.5060, 0.0000],
        [0.0000, 0.5056, 0.0000],
        [0.0000, 0.5039, 0.0000],
        [0.0000, 0.5050, 0.0000],
        [0.0000, 0.5090, 0.0000],
        [0.0000, 0.5115, 0.0000],
        [0.0000, 0.5124, 0.0000],
        [0.0000, 0.5130, 0.0000],
        [0.0000, 0.5135, 0.0000],
        [0.0000, 0.5134, 0.0000],
        [0.0000, 0.5129, 0.0000],
        [0.0000, 0.5114, 0.0000],
        [0.0000, 0.5089, 0.0000],
        [0.0000, 0.5078, 0.0000],
        [0.0000, 0.5078, 0.0000],
        [0.0000, 0.5055, 0.0000],
        [0.0000, 0.5008, 0.0000],
        [0.0000, 0.4940, 0.0000],
        [0.0000, 0.4851, 0.0000],
        [0.0000, 0.4746, 0.0000],
        [0.0000, 0.4627, 0.0000],
        [0.0000, 0.4501, 0.0000],
        [0.0000, 0.4367, 0.0000],
        [0.0000, 0.4244, 0.0000],
        [0.0000, 0.4131, 0.0000],
        [

## New Skeleton

In [9]:

from human_body_prior.body_model.body_model import BodyModel

male_bm_path = "/srv/hays-lab/scratch/sanisetty3/smplx2hml/smplx/SMPLX_NEUTRAL.npz"
male_bm = BodyModel(bm_fname=male_bm_path).to("cuda").eval()

In [24]:

def motionx2positions(
    path, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
):
    pose = torch.Tensor(np.load(path)).to(device)

    if pose.shape[0] > 4000:
        body = male_bm(
            pose_body=pose[:4000, 3 : 3 + 63],
            pose_hand=pose[:4000, 66 : 66 + 90],
            pose_jaw=pose[:4000, 66 + 90 : 66 + 93],
            betas=torch.zeros((4000, 10)).to(device),
            root_orient=pose[:4000, :3],
        )
        joint_loc1 = body.Jtr + pose[:4000, 309 : 309 + 3][:, None, :]
        joint_loc1 = joint_loc1.cpu().numpy()

        body = male_bm(
            pose_body=pose[4000:, 3 : 3 + 63],
            pose_hand=pose[4000:, 66 : 66 + 90],
            pose_jaw=pose[4000:, 66 + 90 : 66 + 93],
            betas=torch.zeros((pose.shape[0] - 4000, 10)).to(device),
            root_orient=pose[4000:, :3],
        )
        joint_loc2 = body.Jtr + pose[4000:, 309 : 309 + 3][:, None, :]
        joint_loc2 = joint_loc2.cpu().numpy()

        return np.concatenate([joint_loc1, joint_loc2], 0)

    body = male_bm(
        pose_body=pose[:, 3 : 3 + 63],
        pose_hand=pose[:, 66 : 66 + 90],
        pose_jaw=pose[:, 66 + 90 : 66 + 93],
        betas=torch.zeros((pose.shape[0], 10)).to(device),
        root_orient=pose[:, :3],
    )
    joint_loc = body.Jtr + pose[:, 309 : 309 + 3][:, None, :]
    joint_loc = joint_loc.cpu().numpy()

    return joint_loc

In [28]:
from core.datasets.kinematics import body_joints_id , hand_joints_id, Skeleton, smplx_raw_body_offsets, smplx_kinematic_chain, Skeleton2
kinematic_chain = smplx_kinematic_chain
n_raw_offsets = torch.from_numpy(smplx_raw_body_offsets)

In [26]:
example_data = np.array(
    motionx2positions(
        "/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/smplx_322/humanml/000021.npy"
    )
)
example_data = example_data.reshape(len(example_data), -1, 3)
example_data = torch.from_numpy(example_data)[:, body_joints_id + hand_joints_id, :]

In [27]:
tgt_skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
tgt_offsets = tgt_skel.get_offsets_joints(example_data[0])

In [21]:
path = '/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/smplx_322/idea400/subset_0056/Walking_And_Snapping_At_The_Same_Time.npy'

In [22]:
pose = torch.Tensor(np.load(path)).to(device)
body = male_bm(
        pose_body=pose[:, 3 : 3 + 63],
        pose_hand=pose[:, 66 : 66 + 90],
        pose_jaw=pose[:, 66 + 90 : 66 + 93],
        betas=torch.zeros((pose.shape[0], 10)).to(device),
        root_orient=pose[:, :3],
)

In [23]:
pose[:, 3 : 3 + 63].shape

torch.Size([240, 63])